# ⚖️ Reliability & Variance Testing

How much can we trust our Routine Scores? In this notebook, we perform a reliability stress test. We analyze the variance within our top-performing stations to ensure that the 'mirroring' effect isn't just a fluke. We're looking for high Routine Scores coupled with low variance, which indicates a stable and predictable behavioral pattern.

### 1. Toolkit and Data
We use standard statistical measures to quantify the stability of our stations, pulling in the habitual metrics from our primary model.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
DATA_DIR = Path("../data/processed")
df = pd.read_csv(DATA_DIR / "habitual_metrics.csv")

### 2. Variance and Standard Error Analysis
We calculate the variance for each station's score components. By comparing the top-tier 'Mirror' stations against the network average, we can determine the 'Separation Ratio'—a measure of how much more reliable these elite stations are compared to typical locations.

In [3]:
avg_rs = df['routine_score'].mean()
std_dev = df['routine_score'].std()
count = len(df)
std_error = std_dev / np.sqrt(count)

top_25_pct = df['routine_score'].quantile(0.75)
mirror_stability = df[df['routine_score'] >= top_25_pct]['routine_score'].std()

### 3. The Reliability Verdict
We conclude with a decision-making framework. If our 'Elite' stations show a significant separation from the mean variance, we can confidently move forward with targeting them. High separation ratios signify a 'Clear' or 'Elite' separation from recreational usage.

In [4]:
ratio = (avg_rs / mirror_stability) if mirror_stability != 0 else 0

print("-" * 50)
print("RELIABILITY STRESS TEST RESULTS:")
print(f"Network Mean RS: {avg_rs:.3f}")
print(f"Standard Error: {std_error:.3f}")
print(f"Mirror Stability (Top 25%): {mirror_stability:.3f}")
print(f"Separation Ratio: {ratio:.2f}x")

if ratio >= 1.5:
    print("DECISION: ELITE SEPARATION")
elif ratio >= 1.25:
    print("DECISION: CLEAR SEPARATION")
else:
    print("DECISION: WEAK SEPARATION")

--------------------------------------------------
RELIABILITY STRESS TEST RESULTS:
Network Mean RS: 0.322
Standard Error: 0.001
Mirror Stability (Top 25%): 0.114
Separation Ratio: 1.42x
DECISION: CLEAR SEPARATION
